# 1. Observations on 1 pixel kernel

# 2. BMENet with maxpool

In [2]:
import os
current_dir = os.getcwd()
print("\n\ncurrent_dir = %s" % current_dir)




current_dir = /home/chen4126/ece60146/HW6


In [ ]:
from DLStudio import *

dls = DLStudio(
#                  dataroot = "/home/kak/ImageDatasets/CIFAR-10/",
                dataroot = "./../data/CIFAR-10/",
                image_size = [32,32],
                path_saved_model = "./saved_model",
                momentum = 0.9,
                learning_rate = 1e-4,
                epochs = 6,
                batch_size = 4,
                classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck'),
                use_gpu = True,
            )

bme_net = dls.BMEnet(dls, skip_connections=True, depth=8)         ## if you want to use skips
#bme_net = skip_con.BMEnet(skip_connections=False, depth=8)         ## if you don't want to use skips

bme_net.load_cifar_10_dataset()

## display network properties
number_of_learnable_params = sum(p.numel() for p in bme_net.parameters() if p.requires_grad)
print("\n\nThe number of learnable parameters in the model: %d" % number_of_learnable_params)

## training and testing
bme_net.run_code_for_training(bme_net, display_images=False)

#import pymsgbox      
#response = pymsgbox.confirm("Finished training.  Start testing on unseen data?") 
#if response == "OK":   

bme_net.run_code_for_testing(bme_net, display_images=False)


### 2.1 Train loss curve

### 2.2 Confusion Matrix

# 3. BMENet with stride

### 3.1 Train loss curve

### 3.2 Confusion Matrix

# 4. Table1: Overall accuracy of 2 models table

# 5. Table2: Per class accuracy of 2 models (10x2 table)

# 6. Observations of Maxpool vs Stride

# 7. Skip Connections with MSCOCO

### 7.1 5x3 images

### 7.2 train loss curve

### 7.3 conf matrix 5x5

### 7.4 overall accuracy

### 7.5 per class accuracy 5x1 table

### 7.6 observations